In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xlrd
list_of_period = [5,10,15,20,25,30]
list_of_k = [1,1.5,2]

In [3]:
# Read the data from the Excel file
xls_file = '/home/vishi/bolinger/cumul_ohlc.xls'
xls = xlrd.open_workbook(xls_file)
sheet = xls.sheet_by_index(0)
# Extract the data
closing_prices = []
for row in range(1, sheet.nrows):
    closing_prices.append(sheet.cell_value(row, 4))
    


In [4]:
def get_avg_std(prices, period):
    avg = np.zeros(len(prices))
    std = np.zeros(len(prices))
    for i in range(period, len(prices)):
        avg[i] = np.mean(prices[i-period:i])
        # sample std
        std[i] = np.std(prices[i-period:i], ddof=1)
    return avg, std



In [5]:
def get_bollinger_bands(prices, period, k):
    avg, std = get_avg_std(prices, period)
    upper_band = avg + k * std
    lower_band = avg - k * std
    return avg, upper_band, lower_band
def profit_basic(prices,avg, upper_band, lower_band):
    flag = 0
    profit = 0
    current_price = 0
    list_of_profit = []
    for i in range(len(prices)):
        if flag == 0:
            if prices[i] < lower_band[i]:
                flag = 1
                current_price = prices[i]
            elif prices[i] > upper_band[i]:
                flag = -1
                current_price = prices[i]
        elif flag == 1:
            if prices[i] > avg[i]:
                flag = 0
                profit += prices[i] - current_price
        elif flag == -1:
            if prices[i] < avg[i]:
                flag = 0
                profit += current_price - prices[i]
        list_of_profit.append(profit)
    if flag == 1:
        profit += prices[-1] - current_price
    elif flag == -1:
        profit += current_price - prices[-1]
    list_of_profit.append(profit)
    return list_of_profit

def plot_profit(list_of_profit, period, k):
    plt.plot(list_of_profit)
    plt.title(f'Profit with period {period} and k {k}')
    plt.xlabel('Time')
    plt.ylabel('Profit')
    plt.show()

list_of_profits_of_diff_strategies = []
for period in list_of_period:
    list_to_append = []
    for k in list_of_k:
        avg, upper_band, lower_band = get_bollinger_bands(closing_prices, period, k)
        list_of_profit = profit_basic(closing_prices, avg, upper_band, lower_band)
        # plot_profit(list_of_profit, period, k)
        list_to_append.append(list_of_profit)
    list_of_profits_of_diff_strategies.append(list_to_append)



    
    
    

In [ ]:
reshaped_list_of_profits = []
for i in range(3):
    for j in range(6):
        reshaped_list_of_profits.append(list_of_profits_of_diff_strategies[j][i])

        

(18, 23626)


In [13]:
# i want to check the hypothesis that which value of period and k gives the best profit
# i will sample out the mins and then i want to take those profit at those days and run a hypothesis test 
# then check if the profit is statistically significant
list_of_better_profits_a_more_b = [] #(a,b)
list_of_better_profits_b_more_a = []
list_of_cant_decide = []
# sampling the mins
import random
from scipy import stats

def sample_mins(lenght):
    # sample randome ints in range of 0 to lenght
    list_of_mins = []
    for i in range(100):
        rand = random.randint(0, lenght-1)
        list_of_mins.append(rand)
    return list_of_mins

def hypothesis_test(i,j,reshaped_list_of_profits, list_of_mins):
    # get the profits at the sampled mins
    profits_a = []
    profits_b = []
    for k in list_of_mins:
        profits_a.append(reshaped_list_of_profits[i][k])
        profits_b.append(reshaped_list_of_profits[j][k])
    # run a t test
    
    t_stat, p_value = stats.ttest_ind(profits_a, profits_b)
    if p_value < 0.05:
        if np.mean(profits_a) > np.mean(profits_b):
            list_of_better_profits_a_more_b.append((i,j))
        else:
            list_of_better_profits_b_more_a.append((i,j))
    else:
        list_of_cant_decide.append((i,j))
    

for i in range(len(reshaped_list_of_profits)):
    for j in range(i+1, len(reshaped_list_of_profits)):
        list_of_mins = sample_mins(len(reshaped_list_of_profits[0]))
        hypothesis_test(i,j,reshaped_list_of_profits, list_of_mins)
print("list_of_better_profits_a_more_b")
print(list_of_better_profits_a_more_b)
print("list_of_better_profits_b_more_a")
print(list_of_better_profits_b_more_a)
print("list_of_cant_decide")
print(list_of_cant_decide)

print(len(list_of_better_profits_a_more_b)+len(list_of_better_profits_b_more_a)+len(list_of_cant_decide))



list_of_better_profits_a_more_b
[(1, 6), (1, 12), (2, 3), (2, 4), (2, 6), (2, 7), (2, 8), (2, 12), (2, 13), (3, 6), (3, 12), (4, 6), (4, 7), (4, 12), (4, 13), (5, 6), (5, 7), (5, 8), (5, 9), (5, 12), (5, 13), (5, 14), (7, 12), (7, 13), (8, 12), (8, 13), (9, 12), (9, 13), (10, 12), (10, 13), (10, 14), (11, 12), (11, 13), (11, 14), (15, 17)]
list_of_better_profits_b_more_a
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (1, 2), (1, 3), (1, 4), (1, 5), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 14), (1, 15), (1, 16), (1, 17), (2, 5), (2, 10), (2, 11), (2, 15), (2, 16), (2, 17), (3, 4), (3, 5), (3, 8), (3, 9), (3, 10), (3, 11), (3, 14), (3, 15), (3, 16), (3, 17), (4, 5), (4, 8), (4, 9), (4, 10), (4, 11), (4, 14), (4, 15), (4, 16), (4, 17), (5, 11), (5, 15), (5, 16), (5, 17), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 13), (6, 14), (6, 15), (6, 16), (6, 17), (7, 8), (7, 9), (7, 10), (7, 11), (7, 1